# Manual curation of fermentative pathways and glycolysis

In [1]:
from reframed import load_cbmodel, save_cbmodel, Protein, GPRAssociation, simplify, Environment
from reframed.io.sbml import parse_gpr_rule
from copy import deepcopy

In [2]:
model = load_cbmodel('../models/RcH10_v1.xml')

<span style="color:red">**Load universal model**

In [3]:
model_uni = load_cbmodel("../models/universe_grampos.xml")

### Fermentative pathways 

<span style="color:red">**Changed below because it showed the wrong stoichiometry for the ferredoxin based reactions**</span>

In [4]:
model.remove_reaction('R_ME1')
model.add_reaction_from_str('R_FNRR: 2 M_fdxrd_c + M_h_c + M_nad_c <-> 2 M_fdxo_2_2_c + M_nadh_c')
model.add_reaction_from_str('R_FNRR2: 2 M_fdxrd_c + M_h_c + M_nadp_c <-> 2 M_fdxo_2_2_c + M_nadph_c')
model.add_reaction_from_str('R_H2td: M_h2_c <-> M_h2_e')

prot = Protein()
prot.genes=['G_WP_015925963_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_FNRR2', gpr)

<span style="color:red">**Changing the cofactor stoichiometry to ensure balanced reactions**</span>

In [5]:
for rxn in model.get_metabolite_reactions("M_fdxrd_c"):
    if abs(model.reactions[rxn].stoichiometry["M_fdxrd_c"])==1:
        print(model.reactions[rxn])
        coeff = model.reactions[rxn].stoichiometry["M_fdxrd_c"]
        coeff_ox = model.reactions[rxn].stoichiometry["M_fdxo_2_2_c"]
        model.reactions[rxn].stoichiometry["M_fdxrd_c"]=coeff*2
        model.reactions[rxn].stoichiometry["M_fdxo_2_2_c"]=coeff_ox*2

R_CO2FO: M_co2_c + M_fdxrd_c + 2.0 M_h_c <-> M_co_c + M_fdxo_2_2_c + M_h2o_c [-inf, 0.0]
R_FNRR3: M_fdxo_2_2_c + M_h2_c --> M_fdxrd_c + 2.0 M_h_c
R_MECDPDH4E: M_2mecdp_c + M_fdxrd_c + M_h_c <-> M_fdxo_2_2_c + M_h2mb4p_c + M_h2o_c
R_VOR2bE: M_3mob_c + M_coa_c + M_fdxo_2_2_c --> M_co2_c + M_fdxrd_c + M_h_c + M_ibcoa_c


### Cofactors in glycolysis

In [6]:
model.add_reaction_from_str('R_PFK_ppi: M_f6p_c + M_ppi_c <-> M_fdp_c + M_h_c + M_pi_c') ## CHANGED from g6p to f6p!!
model.reactions['R_PFK_ppi'].name = 'diphosphate--fructose-6-phosphate 1-phosphotransferase'

prot = Protein()
prot.genes=['G_WP_015925658_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_PFK_ppi', gpr)

In [7]:
model.add_reaction_from_str('R_HEX1_gtp: M_glc__D_c + M_gtp_c <-> M_g6p_c + M_gdp_c + M_h_c')
model.reactions['R_HEX1_gtp'].name = 'GTP dependent hexokinase'

prot = Protein()
prot.genes=['G_WP_015926569_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_HEX1_gtp', gpr)

In [8]:
model.add_reaction_from_str('R_GALK2_gtp: M_a_gal__D_c + M_gtp_c --> M_gal1p_c + M_gdp_c + M_h_c')
model.reactions['R_GALK2_gtp'].name = 'alpha Galactokinase GTP dependent'

prot = Protein()
prot.genes=['G_WP_015926586_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_GALK2_gtp', gpr)

In [9]:
model.add_reaction_from_str('R_GALKr_gtp: M_gal_c + M_gtp_c <-> M_gal1p_c + M_gdp_c + M_h_c')
model.reactions['R_GALKr_gtp'].name = 'Galactokinase GTP dependent'

prot = Protein()
prot.genes=['G_WP_015926586_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_GALKr_gtp', gpr)

In [10]:
model.reactions.R_PFK.gpr.proteins.pop(1)
model.reactions.R_HEX1.gpr.proteins.pop(0)
model.remove_reactions(['R_CD6P', 'R_r0191', 'R_ID6P', 'R_PFK', 'R_GALK2', 'R_GALKr'])

### Mis-annotated transporters leading to false negative essentiality predictions

In [11]:
model.remove_reactions(['R_HEX1', 'R_XYLI2','R_GLUKA_1','R_MNLDHr', 'R_BG_CELLB', 'R_GLCt2', 'R_BGLA'])
model.reactions.R_CEPA.reversible=False
model.reactions.R_CEPA.lb=0

### Fix false positive essentiality for xylose isomerase 

In [12]:
p1 = Protein()
p1.genes=['G_WP_015926769_1'] # Ccel_3429 (original xylose isomerase)
p2 = Protein()
p2.genes=['G_WP_012634944_1'] # Ccel_0500 (alternative xylose isomerase)
gpr = GPRAssociation()
gpr.proteins=[p1, p2]
model.set_gpr_association('R_XYLI1', gpr)

### Other mis-annotated suger transporters

In [13]:
model.remove_reactions(['R_ARBabcpp', 'R_GLCabcpp', 'R_XYLabcpp'])

rxns = [rxn for rxn in model.get_reactions_by_gene('G_WP_041706401_1') if len(model.reactions[rxn].get_genes())==1]
model.remove_reactions(rxns)

model.add_reaction_from_str('R_GALabc: M_atp_c + M_gal_e + M_h2o_c <-> M_adp_c + M_gal_c + M_h_c + M_pi_c')
gpr = deepcopy(model.reactions.R_GALabcpp.gpr)
model.set_gpr_association('R_GALabc', gpr)

pts_rxns = [rxn for rxn in [rxn for rxn in model.reactions if rxn.endswith('pts')]]
model.remove_reactions(pts_rxns)

model.add_reaction_from_str('R_GALUi: M_g1p_c + M_h_c + M_utp_c <-> M_ppi_c + M_udpg_c')
prot = Protein()
prot.genes=['G_WP_015926747_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_GALUi', gpr)

### Remove ABC transporters using both cytosol and periplasm

In [14]:
duplicated = [r_id for r_id in model.reactions if r_id.endswith('abcpp') and r_id[:-2] in model.reactions]

for r_id in duplicated:
    gpr1 = model.reactions[r_id].gpr
    gpr2 = model.reactions[r_id[:-2]].gpr
    gpr_str = f'({gpr1}) or ({gpr2})'

    gpr = parse_gpr_rule(gpr_str)
    model.set_gpr_association(r_id[:-2], gpr)
    
model.remove_reactions(duplicated)

### <span style="color:red"> Energy balancing

In [15]:
model.add_reaction_from_str("R_ATPS4r: M_adp_c + M_pi_c + 4.0 M_h_e --> M_atp_c + 3.0 M_h_c + M_h2o_c")
model.remove_reaction("R_ATPS4rpp")

genes_dict = {
"atpA": ["G_Ccel_0270"],
"atpB": ["G_WP_012634718_1"],
"atpC": ["G_WP_012634725_1"],
"atpD": ["G_WP_012634724_1"],
"atpE": ["G_WP_012634719_1"],
"atpF": ["G_WP_012634720_1"],
"atpG": ["G_WP_012634723_1"],
"atpH": ["G_WP_012634721_1"]
}

proteins_list = []

for genes in genes_dict.values():
    prot = Protein()
    prot.genes=genes
    proteins_list.append(prot)
    
gpr = GPRAssociation()    
gpr.proteins=proteins_list
model.set_gpr_association('R_ATPS4r', gpr)

model.reactions.R_ATPS4r.reversible=True
model.reactions.R_ATPS4r.lb=-1000

In [16]:
model.reactions.R_FNRR3.reversible=True
model.reactions.R_FNRR3.lb=-1000

# This is more likely to be https://www.uniprot.org/uniprotkb?query=WP_015926678.1, and lead to too high production of acetate
model.remove_reaction("R_ALDD2y")
model.reactions.R_ACALD.ub=0

model.remove_reaction("R_ACt2r_1") ## Replaced this reactions because it doesn't have any genetic evidence and is far less common than ACt2r http://bigg.ucsd.edu/universal/reactions/ACt2r_1
model.add_reaction(deepcopy(model_uni.reactions.R_ACt2r))
model.reactions.R_ACt2r.reversible=True
model.reactions.R_ACt2r.lb=-1000

### <span style="color:red"> Pyruvate exchange

In [17]:
model.add_metabolite(deepcopy(model_uni.metabolites.M_pyr_e))
model.add_reaction(deepcopy(model_uni.reactions.R_PYRt2))
model.add_reaction(deepcopy(model_uni.reactions.R_EX_pyr_e))
model.reactions.R_PYRt2.reversible=True
model.reactions.R_PYRt2.lb=-1000


### Remove ubiquinone reactions (including hydrogenase that converts h2 back to h+ with ubiquinone) 

In [18]:
model.remove_reactions(model.get_metabolite_reactions("M_q8_c"))

## Remove blocked reactions and orphaned genes

In [19]:
model.update()

In [20]:
Environment.complete(model, inplace=True)
simplify(model, inplace=True)

## Save model

In [21]:
save_cbmodel(model, '../models/RcH10_v2.xml')

### <span style="color:red"> Fixing imbalanced reactions

In [22]:
%run ../scripts/fixing_mass_imbalances.py

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-14
Number of imbalanced reactions before: 320
Number of imbalanced reactions now: 190
